In [2]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import random
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_columns', None)
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from math import floor
import os

### Loading the combined data sets

In [ ]:
df=pd.read_csv(r'compiled_data\csh_combined_101.csv')

In [69]:
cat_list=[ 'sensor','location','location2','message','senor_type','lastSensorEventHours','lastSensorDayOfWeek','prevDominantSensor1','prevDominantSensor2','lastSensorID','lastSensorLocation','lastMotionLocation']																	
num_list=[ 'lastSensorEventSeconds','windowDuration','timeSinceLastSensorEvent','complexity','activityChange','areaTransitions','numDistinctSensors','sensorCount-Bathroom','sensorCount-Bedroom','sensorCount-Chair','sensorCount-DiningRoom','sensorCount-Hall','sensorCount-Ignore','sensorCount-Kitchen','sensorCount-LivingRoom','sensorCount-Office','sensorCount-OutsideDoor','sensorCount-WorkArea','sensorElTime-Bathroom','sensorElTime-Bedroom','sensorElTime-Chair','sensorElTime-DiningRoom','sensorElTime-Hall','sensorElTime-Ignore','sensorElTime-Kitchen','sensorElTime-LivingRoom','sensorElTime-Office','sensorElTime-OutsideDoor','sensorElTime-WorkArea']
target_mapper={'Other_Activity': 'Other', 'Entertain_Guests': 'Entertain_Guests', 'Personal_Hygiene': 'Washroom', 'Cook_Breakfast': 'Prep_for_meal', 'Watch_TV': 'TV', 'Sleep': 'Sleep', 'Work_On_Computer': 'Work', 'Dress': 'Getting_ready', 'Groom': 'Getting_ready', 'Cook_Dinner': 'Prep_for_meal', 'Toilet': 'Washroom', 'Wash_Dishes': 'Prep_for_meal', 'Cook': 'Prep_for_meal', 'Cook_Lunch': 'Prep_for_meal', 'Drink': 'Exclude', 'Wash_Breakfast_Dishes': 'Prep_for_meal', 'Work': 'Work', 'Read': 'Read', 'Bathe': 'Washroom', 'Phone': 'Exclude', 'Relax': 'Relax', 'Work_At_Table': 'Work', 'Wash_Dinner_Dishes': 'Prep_for_meal', 'Leave_Home': 'Leave_Home', 'Eat_Breakfast': 'Eat', 'Enter_Home': 'Enter_Home', 'Bed_Toilet_Transition': 'Exclude', 'Sleep_Out_Of_Bed': 'Sleep', 'Morning_Meds': 'Exclude', 'Wash_Lunch_Dishes': 'Prep_for_meal', 'Eat_Dinner': 'Eat', 'Evening_Meds': 'Exclude', 'Eat_Lunch': 'Eat', 'Eat': 'Eat', 'Take_Medicine': 'Exclude', 'Step_Out': 'Leave_Home', 'Work_At_Desk': 'Work', 'Laundry': 'Exclude', 'Nap': 'Exclude', 'Go_To_Sleep': 'Exclude', 'Wake_Up': 'Exclude', 'Exercise': 'Exclude', 'r1.Sleep': 'Exclude', 'r2.Personal_Hygiene': 'Exclude', 'r1.Cook_Breakfast': 'Exclude', 'r2.Dress': 'Exclude', 'r2.Eat_Breakfast': 'Exclude'}

In [36]:
df['target']=df['activity'].apply(lambda x: target_mapper[x])
df=df[df['target']!='Exclude']
df=df[['date_time', 'sensor', 'location', 'location2', 'message', 'senor_type', 'lastSensorEventHours', 'lastSensorEventSeconds', 'lastSensorDayOfWeek', 'windowDuration', 'timeSinceLastSensorEvent', 'prevDominantSensor1', 'prevDominantSensor2', 'lastSensorID', 'lastSensorLocation', 'lastMotionLocation', 'complexity', 'activityChange', 'areaTransitions', 'numDistinctSensors', 'sensorCount-Bathroom', 'sensorCount-Bedroom', 'sensorCount-Chair', 'sensorCount-DiningRoom', 'sensorCount-Hall', 'sensorCount-Ignore', 'sensorCount-Kitchen', 'sensorCount-LivingRoom', 'sensorCount-Office', 'sensorCount-OutsideDoor', 'sensorCount-WorkArea', 'sensorElTime-Bathroom', 'sensorElTime-Bedroom', 'sensorElTime-Chair', 'sensorElTime-DiningRoom', 'sensorElTime-Hall', 'sensorElTime-Ignore', 'sensorElTime-Kitchen', 'sensorElTime-LivingRoom', 'sensorElTime-Office', 'sensorElTime-OutsideDoor', 'sensorElTime-WorkArea','room', 'target']]
df[cat_list]=df[cat_list].astype('str')

In [49]:
df[['date_time']+stratify_list]

,date_time,location,location2,message,senor_type,lastSensorEventHours,lastSensorDayOfWeek,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,target,room
0,2012-07-20 10:41:05,OutsideDoor,Entry,OFF,Control4-Motion,10.0,4.0,0.0,0.0,9.0,9.0,9.0,Leave_Home,csh101
1,2012-07-20 10:42:58,Ignore,FrontDoorTemp,77,Control4-Temperature,10.0,4.0,0.0,0.0,5.0,5.0,9.0,Leave_Home,csh101
2,2012-07-20 10:43:02,Ignore,Ignore,6,Control4-LightSensor,10.0,4.0,0.0,0.0,5.0,5.0,9.0,Leave_Home,csh101
3,2012-07-20 10:43:02,OutsideDoor,Entry,ON,Control4-Motion,10.0,4.0,0.0,0.0,9.0,9.0,9.0,Leave_Home,csh101
4,2012-07-20 10:43:04,Ignore,Ignore,48,Control4-LightSensor,10.0,4.0,0.0,0.0,5.0,5.0,9.0,Leave_Home,csh101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371697,2011-08-14 23:46:15,Ignore,Ignore,0,Control4-LightSensor,23.0,6.0,0.0,0.0,5.0,5.0,1.0,Other,csh104
1371698,2011-08-14 23:46:41,Ignore,Ignore,0,Control4-LightSensor,23.0,6.0,0.0,0.0,5.0,5.0,1.0,Other,csh104
1371699,2011-08-14 23:46:42,Ignore,Ignore,0,Control4-LightSensor,23.0,6.0,0.0,0.0,5.0,5.0,1.0,Other,csh104
1371700,2011-08-14 23:47:13,Ignore,BathroomTemp,23,Control4-Temperature,23.0,6.0,0.0,0.0,5.0,5.0,1.0,Other,csh104


### Undersampling Other target variable to maintain the balance of the distribution

In [53]:
other_df=df[df['target']=='Other']
print(df.shape[0])
print(other_df.shape[0])
X=other_df.drop(['target'],axis=1)
y=other_df['target']
X_keep, X_exclude, y_keep, y_exclude = train_test_split(X, y, test_size=.5, random_state=42,stratify=other_df['room'])
other_df=pd.concat([X_keep,y_keep],axis=1)
print(other_df.shape[0])
exclude_df=pd.concat([X_exclude,y_exclude],axis=1)
df=df[df['target']!='Other']
df=pd.concat([df,other_df])
print(df.shape[0])

1318594
474149
237074
1081519


### Splitting into train and test

In [60]:
X=df.drop(['target'],axis=1)
y=df['target']
X_dum, X_valid, y_dum, y_valid = train_test_split(X, y, test_size=0.25, random_state=42,stratify=df[['target','room']])
X_train, X_test, y_train, y_test = train_test_split(X_dum, y_dum, test_size=0.33, random_state=42,stratify=pd.concat([X_dum['lastSensorEventHours'],y_dum],axis=1))

In [65]:
X_train.to_csv(r'model_dataset\csh_X_train_'+ str(i+1)+'.csv',index=None)
X_test.to_csv(r'model_dataset\csh_X_test_'+ str(i+1)+'.csv',index=None)
X_valid.to_csv(r'model_dataset\csh_X_valid_'+ str(i+1)+'.csv',index=None)
y_train.to_csv(r'model_dataset\csh_y_train_'+ str(i+1)+'.csv',index=None)
y_test.to_csv(r'model_dataset\csh_y_test_'+ str(i+1)+'.csv',index=None)
y_valid.to_csv(r'model_dataset\csh_y_valid_'+ str(i+1)+'.csv',index=None)

In [71]:
for i in range(2,13):
    df=pd.read_csv(r'compiled_data\csh_combined_'+str(i+1)+'.csv')
    df['target']=df['activity'].apply(lambda x: target_mapper[x])
    df=df[df['target']!='Exclude']
    df=df[['date_time', 'sensor', 'location', 'location2', 'message', 'senor_type', 'lastSensorEventHours', 'lastSensorEventSeconds', 'lastSensorDayOfWeek', 'windowDuration', 'timeSinceLastSensorEvent', 'prevDominantSensor1', 'prevDominantSensor2', 'lastSensorID', 'lastSensorLocation', 'lastMotionLocation', 'complexity', 'activityChange', 'areaTransitions', 'numDistinctSensors', 'sensorCount-Bathroom', 'sensorCount-Bedroom', 'sensorCount-Chair', 'sensorCount-DiningRoom', 'sensorCount-Hall', 'sensorCount-Ignore', 'sensorCount-Kitchen', 'sensorCount-LivingRoom', 'sensorCount-Office', 'sensorCount-OutsideDoor', 'sensorCount-WorkArea', 'sensorElTime-Bathroom', 'sensorElTime-Bedroom', 'sensorElTime-Chair', 'sensorElTime-DiningRoom', 'sensorElTime-Hall', 'sensorElTime-Ignore', 'sensorElTime-Kitchen', 'sensorElTime-LivingRoom', 'sensorElTime-Office', 'sensorElTime-OutsideDoor', 'sensorElTime-WorkArea','room', 'target']]
    df[cat_list]=df[cat_list].astype('str')

    other_df=df[df['target']=='Other']
    print(df.shape[0])
    print(other_df.shape[0])
    X=other_df.drop(['target'],axis=1)
    y=other_df['target']
    X_keep, X_exclude, y_keep, y_exclude = train_test_split(X, y, test_size=.5, random_state=42,stratify=other_df['room'])
    other_df=pd.concat([X_keep,y_keep],axis=1)
    print(other_df.shape[0])
    exclude_df=pd.concat([X_exclude,y_exclude],axis=1)
    df=df[df['target']!='Other']
    df=pd.concat([df,other_df])
    print(df.shape[0])
    X=df.drop(['target'],axis=1)
    y=df['target']
    X_dum, X_valid, y_dum, y_valid = train_test_split(X, y, test_size=0.25, random_state=42,stratify=df[['target','room']])
    X_train, X_test, y_train, y_test = train_test_split(X_dum, y_dum, test_size=0.33, random_state=42,stratify=y_dum)
    X_train.to_csv(r'model_dataset\csh_X_train_'+ str(i+1)+'.csv',index=None)
    X_test.to_csv(r'model_dataset\csh_X_test_'+ str(i+1)+'.csv',index=None)
    X_valid.to_csv(r'model_dataset\csh_X_valid_'+ str(i+1)+'.csv',index=None)
    y_train.to_csv(r'model_dataset\csh_y_train_'+ str(i+1)+'.csv',index=None)
    y_test.to_csv(r'model_dataset\csh_y_test_'+ str(i+1)+'.csv',index=None)
    y_valid.to_csv(r'model_dataset\csh_y_valid_'+ str(i+1)+'.csv',index=None)

1008356
397500
198750
809606


C:\Users\daksh\AppData\Local\Temp\ipykernel_12700\2578831134.py:2: DtypeWarning: Columns (47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'compiled_data\csh_combined_'+str(i+1)+'.csv')


1008693
395295
197647
811045
970226
266410
133205
837021
974387
347173
173586
800800
1007608
301264
150632
856976
992254
263953
131976
860277
1293610
281983
140991
1152618
995977
360628
180314
815663


C:\Users\daksh\AppData\Local\Temp\ipykernel_12700\2578831134.py:2: DtypeWarning: Columns (46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'compiled_data\csh_combined_'+str(i+1)+'.csv')


984902
353173
176586
808315
1018140
559948
279974
738166
357560
158619
79309
278250


In [78]:
i=0
X_train=pd.read_csv(r"model_dataset\csh_X_train_"+str(i+1)+".csv")
for i in range(13):
    X_train=pd.concat([X_train,pd.read_csv(r"model_dataset\csh_X_train_"+str(i+1)+".csv")])
X_train.to_csv(r'model_dataset\csh_X_train.csv',index=None)
location_counts=pd.DataFrame(X_train['location'].value_counts()).reset_index()
location2_counts=pd.DataFrame(X_train['location2'].value_counts()).reset_index()
message_counts=pd.DataFrame(X_train['message'].value_counts()).reset_index()
senor_type_counts=pd.DataFrame(X_train['senor_type'].value_counts()).reset_index()
sensor_counts=pd.DataFrame(X_train['sensor'].value_counts()).reset_index()

i=0
X_test=pd.read_csv(r"model_dataset\csh_X_test_"+str(i+1)+".csv")
for i in range(13):
    X_test=pd.concat([X_test,pd.read_csv(r"model_dataset\csh_X_test_"+str(i+1)+".csv")])
X_test.to_csv(r'model_dataset\csh_X_test.csv',index=None)
location_counts=location_counts.merge(pd.DataFrame(X_test['location'].value_counts()).reset_index(),how='outer',on='index')
location2_counts=location2_counts.merge(pd.DataFrame(X_test['location2'].value_counts()).reset_index(),how='outer',on='index')
message_counts=message_counts.merge(pd.DataFrame(X_test['message'].value_counts()).reset_index(),how='outer',on='index')
senor_type_counts=senor_type_counts.merge(pd.DataFrame(X_test['senor_type'].value_counts()).reset_index(),how='outer',on='index')
sensor_counts=sensor_counts.merge(pd.DataFrame(X_test['sensor'].value_counts()).reset_index(),how='outer',on='index')

i=0
X_valid=pd.read_csv(r"model_dataset\csh_X_valid_"+str(i+1)+".csv")
for i in range(13):
    X_valid=pd.concat([X_valid,pd.read_csv(r"model_dataset\csh_X_valid_"+str(i+1)+".csv")])
X_valid.to_csv(r'model_dataset\csh_X_valid.csv',index=None)
location_counts=location_counts.merge(pd.DataFrame(X_valid['location'].value_counts()).reset_index(),how='outer',on='index')
location2_counts=location2_counts.merge(pd.DataFrame(X_valid['location2'].value_counts()).reset_index(),how='outer',on='index')
message_counts=message_counts.merge(pd.DataFrame(X_valid['message'].value_counts()).reset_index(),how='outer',on='index')
senor_type_counts=senor_type_counts.merge(pd.DataFrame(X_valid['senor_type'].value_counts()).reset_index(),how='outer',on='index')
sensor_counts=sensor_counts.merge(pd.DataFrame(X_valid['sensor'].value_counts()).reset_index(),how='outer',on='index')

location_counts.to_csv(r'model_dataset\location_variable_counts.csv',index=None)
location2_counts.to_csv(r'model_dataset\location2_variable_counts.csv',index=None)
message_counts.to_csv(r'model_dataset\message_variable_counts.csv',index=None)
senor_type_counts.to_csv(r'model_dataset\senor_type_variable_counts.csv',index=None)
sensor_counts.to_csv(r'model_dataset\sensor_variable_counts.csv',index=None)

i=0
y_train=pd.read_csv(r"model_dataset\csh_y_train_"+str(i+1)+".csv")
for i in range(13):
    y_train=pd.concat([y_train,pd.read_csv(r"model_dataset\csh_y_train_"+str(i+1)+".csv")])
y_train.to_csv(r'model_dataset\csh_y_train.csv',index=None)

target_counts=pd.DataFrame(y_train.value_counts()).reset_index()

i=0
y_test=pd.read_csv(r"model_dataset\csh_y_test_"+str(i+1)+".csv")
for i in range(13):
    y_test=pd.concat([y_test,pd.read_csv(r"model_dataset\csh_y_test_"+str(i+1)+".csv")])
y_test.to_csv(r'model_dataset\csh_y_test.csv',index=None)
target_counts=target_counts.merge(pd.DataFrame(y_test.value_counts()).reset_index(),how='outer',on='target')

i=0
y_valid=pd.read_csv(r"model_dataset\csh_y_valid_"+str(i+1)+".csv")
for i in range(13):
    y_valid=pd.concat([y_valid,pd.read_csv(r"model_dataset\csh_y_valid_"+str(i+1)+".csv")])
y_valid.to_csv(r'model_dataset\csh_y_valid.csv',index=None)
target_counts=target_counts.merge(pd.DataFrame(y_valid.value_counts()).reset_index(),how='outer',on='target')

target_counts.to_csv(r'model_dataset\target_variable_counts.csv',index=None)

In [79]:
X_train

,sensor,location,location2,message,senor_type,lastSensorEventHours,lastSensorDayOfWeek,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,lastSensorEventSeconds,windowDuration,timeSinceLastSensorEvent,complexity,activityChange,areaTransitions,numDistinctSensors,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
0,M008,Kitchen,Kitchen,OFF,Control4-Motion,18.000000,4.000000,6.000000,5.000000,6.000000,6.000000,6.000000,67236.000000,59.000000,0.000000,0.783777,0.389831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.170000,26.180000,0.000000,0.000000,0.000000,0.000000,4631.252905,4195.487419,86400.000000,118.449243,86400.000000,0.402761,0.000000,119.516528,86400.000000,3770.352859,4189.263011
1,MA013,LivingRoom,LivingRoom,ON,Control4-MotionArea,16.000000,3.000000,0.000000,5.000000,7.000000,7.000000,7.000000,58646.000000,61.000000,0.000000,1.403674,0.573770,1.000000,0.000000,20.090000,0.000000,0.000000,0.000000,0.000000,9.150000,0.000000,5.110000,0.000000,0.000000,0.000000,13.478419,17156.081468,86400.000000,19501.182607,86400.000000,8.475715,19425.574446,0.000000,86400.000000,265.409013,86400.000000
2,M008,LivingRoom,Chair,ON,Control4-Motion,9.000000,1.000000,6.000000,6.000000,7.000000,7.000000,7.000000,34958.000000,320.000000,8.000000,1.241946,0.181250,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.300000,0.000000,22.240000,0.000000,4.810000,0.000000,3784.110391,3939.652611,86400.000000,350.534183,86400.000000,265.942078,350.704024,0.000000,86400.000000,308.349595,86400.000000
3,LS022,Ignore,Ignore,36,Control4-LightSensor,14.000000,5.000000,6.000000,6.000000,5.000000,5.000000,6.000000,51459.000000,37.000000,1.000000,0.918296,0.594595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.430000,22.920000,0.000000,0.000000,0.000000,0.000000,4332.595631,456.210724,1743.778791,86400.000000,86400.000000,0.000000,1.003861,437.759688,86400.000000,4441.825389,1750.291579
4,LS020,Ignore,Ignore,9,Control4-LightSensor,12.000000,3.000000,6.000000,5.000000,5.000000,5.000000,6.000000,45166.000000,69.000000,2.000000,0.468996,0.579710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.300000,31.050000,0.000000,0.000000,0.000000,0.000000,1521.735071,6386.382299,86400.000000,181.398060,86400.000000,0.000000,2.179429,196.038538,86400.000000,1980.381580,1454.150398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139815,LS009,Ignore,Ignore,05,Control4-LightSensor,16.000000,0.000000,5.000000,7.000000,5.000000,5.000000,7.000000,58266.000000,2227.000000,12.000000,0.996792,0.710822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.100000,0.000000,16.250000,0.000000,0.000000,0.000000,5643.498956,3756.011759,86400.000000,86400.000000,86400.000000,0.000000,3635.045358,12.423270,86400.000000,3764.466846,86400.000000
139816,MA010,Bedroom,Bedroom,ON,Control4-MotionArea,22.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,82489.000000,556.000000,1.000000,1.038921,0.050360,6.000000,0.000000,0.000000,27.570000,0.000000,0.000000,0.000000,2.090000,0.000000,2.430000,0.000000,2.260000,0.000000,750.874976,0.000000,86400.000000,86400.000000,86400.000000,374.295954,1568.456759,549.309264,86400.000000,527.154053,86400.000000
139817,MA009,LivingRoom,Chair,OFF,Control4-MotionArea,8.000000,1.000000,1.000000,1.000000,7.000000,7.000000,7.000000,31195.000000,21.000000,0.000000,2.050195,0.714286,6.000000,0.000000,1.250000,6.880000,0